In [4]:
import pandas as pd

integrated_og_df = pd.read_csv('../outputs/integrated_ocean_plastics.csv')
integrated_og_df.describe()

,Year,mp/kg dw,log_concentration
count,2597.000000,2.729000e+03,2729.000000
mean,2018.641894,1.373535e+04,0.852433
std,0.500755,7.051343e+05,0.829072
min,2013.000000,0.000000e+00,0.000000
25%,2018.000000,7.231181e-01,0.236315
50%,2019.000000,3.744196e+00,0.676163
75%,2019.000000,1.875361e+01,1.295646
max,2019.000000,3.683603e+07,7.566273


In [10]:
integrated_og_df.columns

Index(['sample_id', 'Continent', 'Country', 'region', 'System', 'Waterbody',
       'Zone Area', 'Tidal_Zone', 'Test_Area', 'Year', 'Date', 'mp/kg dw',
       'log_concentration', 'Size_class', 'Standardized_Shapes',
       'Dominant_Size', 'dominant_color', 'source', 'environment_type',
       'location', 'concentration_level'],
      dtype='object')

In [11]:
xb_color_count = pd.read_csv("../data/Xialiao Beach color count.csv")
lb_color_count = pd.read_csv("../data/Longmen Beach color count.csv")

In [12]:
print(xb_color_count.columns)
print(lb_color_count.columns)

Index(['Date_YYYY-MM-DD', 'Country_Region', 'Location_name', 'Location_lat',
       'Location_lon', 'Transect', 'Position', 'Size_min_mm', 'Size_max_mm',
       'Size_class', 'no_color', 'black', 'grey', 'red_pink',
       'orange_brown_yellow', 'green', 'blue', 'purple'],
      dtype='object')
Index(['Date_YYYY-MM-DD', 'Country_Region', 'Location_name', 'Location_lat',
       'Location_lon', 'Transect', 'Position', 'Size_min_mm', 'Size_max_mm',
       'Size_class', 'no_color', 'black', 'grey', 'red_pink',
       'orange_brown_yellow', 'green', 'blue', 'purple'],
      dtype='object')


In [13]:
combined_color_count = pd.concat([xb_color_count, lb_color_count], ignore_index=True)

# Check the shape of the combined dataframe
print(f"Xialiao Beach shape: {xb_color_count.shape}")
print(f"Longmen Beach shape: {lb_color_count.shape}")
print(f"Combined dataframe shape: {combined_color_count.shape}")

# Preview the combined dataframe
combined_color_count.head()

Xialiao Beach shape: (966, 18)
Longmen Beach shape: (1626, 18)
Combined dataframe shape: (2592, 18)


,Date_YYYY-MM-DD,Country_Region,Location_name,Location_lat,Location_lon,Transect,Position,Size_min_mm,Size_max_mm,Size_class,no_color,black,grey,red_pink,orange_brown_yellow,green,blue,purple
0,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,1,1,5,microplastics,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,2,1,5,microplastics,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,3,1,5,microplastics,76.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
3,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,4,1,5,microplastics,56.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,5,1,5,microplastics,179.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0


In [14]:
# Save the combined dataframe to a new CSV file
combined_color_count.to_csv('../outputs/combined_beach_color_count.csv', index=False)

In [15]:
pd.read_csv('../outputs/combined_beach_color_count.csv')

,Date_YYYY-MM-DD,Country_Region,Location_name,Location_lat,Location_lon,Transect,Position,Size_min_mm,Size_max_mm,Size_class,no_color,black,grey,red_pink,orange_brown_yellow,green,blue,purple
0,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,1,1,5,microplastics,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,2,1,5,microplastics,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,3,1,5,microplastics,76.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
3,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,4,1,5,microplastics,56.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2018-04-25,Taiwan,Xialiao_Beach,25.21469,121.65406,A,5,1,5,microplastics,179.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,2019-11-27,Taiwan,Longmen_Beach,25.02951,121.93550,C,29,5,25,mesoplastics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2588,2019-11-27,Taiwan,Longmen_Beach,25.02951,121.93550,C,30,5,25,mesoplastics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2589,2019-11-27,Taiwan,Longmen_Beach,25.02951,121.93550,C,31,5,25,mesoplastics,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2590,2019-11-27,Taiwan,Longmen_Beach,25.02951,121.93550,C,32,5,25,mesoplastics,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
